In [ ]:
#pip install --quiet --upgrade rich

In [ ]:
#pip install --quiet --upgrade streamlit

In [ ]:
#pip install --quiet --upgrade datasets

In [ ]:
#pip install --quiet --upgrade sentence_transformers

In [ ]:
#pip install --quiet --upgrade torchvision

In [ ]:
#pip install --quiet --upgrade torch

In [ ]:
#pip install --quiet --upgrade pinecone-client

In [ ]:
#pip install --quiet --upgrade pinecone-datasets

In [ ]:
#pip install --quiet --upgrade pinecone-text

In [ ]:
#pip install --quiet --upgrade tqdm

In [ ]:
#pip install --quiet --upgrade tiktoken

In [ ]:
#pip install --quiet --upgrade deepface

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#pip install tf-keras==2.17.0

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os
import time
import torch

In [ ]:
from UDCUtils import UDCUtils

utils = UDCUtils()

In [ ]:
pinecone_api_key = utils.get_pinecone_api_key()
#print(type(pinecone_api_key))

if pinecone_api_key==None:
    pinecone_api_key = "a85f0b7c-6ce3-458d-aa18-168f39e4420b" ##default key

In [ ]:
print(pinecone_api_key)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
import datasets

#datasets.get_dataset_infos(path="quora")

datasets.list_datasets()

In [ ]:
dataset = load_dataset('quora', split='train[:10000]', trust_remote_code=True)

In [ ]:
dataset[:5]

In [ ]:
#read all questions in a list
questions = []

for record in dataset["questions"]:
    questions.extend(record['text'])

#convert list into set of unique questions
question_lst = list(set(questions))

In [ ]:
print('-'*50)
print("\n".join(question_lst[:10]))
print('-'*50)
print(f"Sample questions above out of total No of questions:{len(question_lst)}")

In [ ]:
### checking if we have access to GPU by checking value of cuda

device = 'cuda' if torch.cuda.is_available() else "cpu"

if device != 'cuda':
    print("No cuda available")

In [ ]:
### using transformer model all-MiniLM-L6-v2 that maps vector space to 384 dimensional space
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [ ]:
query = "where does the queen of england live?"
xq = model.encode(query)
print(xq.shape)

In [ ]:
pinecone_obj = Pinecone(api_key=pinecone_api_key)
dl_ai_index_name = utils.create_dlai_index_name('dl-ai')
print(dl_ai_index_name)

In [ ]:
print(utils.get_dlapi_key())
print(len(utils.get_dlapi_key()))

In [ ]:
print(utils.get_dlapi_key()[-36:-3])

In [ ]:
dl_ai_index_name = dl_ai_index_name[:-3]

if dl_ai_index_name in pinecone_obj.list_indexes():
    pinecone_obj.delete_index(dl_ai_index_name)

print(dl_ai_index_name)

In [ ]:
pinecone_obj.create_index(
    name=dl_ai_index_name,
    dimension=model.get_sentence_embedding_dimension(),
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

index = pinecone_obj.Index(dl_ai_index_name)
print(index)

In [ ]:
## create embeddings and upsert to pinecone
batch_size = 200
vector_limit = 1000

questions = question_lst[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    i_end = min(i+batch_size, vector_limit)
    ids = [str(x) for x in range(i, i_end)]
    metadata = [{'text':text} for text in questions[i:i_end]]
    xq = model.encode(questions[i:i_end])
    records = zip(ids, xq, metadata)
    index.upsert(vectors=records)
    

In [ ]:
index.describe_index_stats()

In [ ]:
## run your query
def run_query(query):
    embedding = model.encode(query).tolist()
    results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
    for result in results["matches"]:
        print(result)

In [ ]:
run_query("Why is baking soda added to cake batter?")